In [ ]:
%load_ext autoreload
%autoreload 2
import theseus as th
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
torch.autograd.set_detect_anomaly(True)
from tqdm import tqdm
from typing import List

import numpy as np

from spline_generation import generate_batch_of_splines
from spline_dataloader import Spline_2D_Dataset

from torch.utils.data import DataLoader

# Defining NN model

In [ ]:
# simple MLP model for initial tests
class SimpleNN(nn.Module):
    def __init__(self, in_size, out_size, hid_size=30, use_offset=False):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(in_size, hid_size, bias=False),
            nn.ReLU(),
            nn.Linear(hid_size, hid_size, bias=False),
            nn.ReLU(),
            nn.Linear(hid_size, out_size, bias=False),
        )

    def forward(self, state_):
        return self.fc(state_)

# Generating data
Generating 2d data - moving along spline curve on a 2D plane.

Generatint batch of data for training

In [ ]:
path_to_splines = "./splines/"

B = 4 # number of splines in batch
n_batch = 1 # number of batches to generate
n_control_points = 5 # number of control points in spline. Matches number of poses in optimization problem
n_pts_spline_segment = 100 # number of points for each spline segment during interpolation

In [ ]:
generate_batch_of_splines(path_to_splines, B*n_batch, n_control_points, n_pts_spline_segment)

In [ ]:
window = 100
dataset = Spline_2D_Dataset(path_to_splines,window=window)

In [ ]:
plt.figure(figsize=(10,8))
X, y, traj, gt_poses, gt_velocity = dataset.__getitem__(3)
for i in range(X.shape[0]):
    plt.plot(y[i,:,0], y[i,:,1])
    plt.quiver(y[i,::20,0],y[i,::20,1],y[i,::20,2],y[i,::20,3])

plt.title("GT trajectories from slices of a single track")
# plt.axis('equal')
plt.xlabel('x_sensor_frame')
plt.ylabel('y_sensor_frame')

plt.grid()


plt.figure(figsize=(10,8))
plt.title("original spline curve")
plt.plot(traj[:,0],traj[:,1])
plt.scatter(gt_poses[:,0], gt_poses[:,1])
plt.quiver(gt_poses[:,0], gt_poses[:,1],gt_poses[:,2], gt_poses[:,3])
plt.grid()
plt.axis('equal')


plt.show()

# Double integration method

Let's implement differentiable double integration method. Input is IMU data (ACC + GYRO) and output is piee of trajectory with a given $x_0$ and $v_0$ abounday conditions. 

In [ ]:
# consider one IMU slice as an example
imu_slice = X[1]
plt.figure()
plt.title('IMU slice in sensor frame')
plt.plot(imu_slice[:,0],label='acc_x')
plt.plot(imu_slice[:,1],label='acc_y')
plt.plot(imu_slice[:,2],label='omega_z')
plt.grid()
plt.legend()
plt.show()


In [ ]:
# piece of trajectory we want to reconstruct
# [x,y, cos, sin]
traj_segment = y[1]
plt.figure()
plt.plot(traj_segment[:,0],traj_segment[:,1])
plt.quiver(traj_segment[::10,0], traj_segment[::10,1], traj_segment[::10,2],traj_segment[::10,3])
plt.grid()
plt.axis('equal')
plt.show()

In [ ]:
# def double_integrate(imu_slice, v0=torch.tensor([0,0], dtype=torch.float32)):
    
#     B,S,W,C = imu_slice.shape
#     result = torch.zeros((B,S,W,4)) # result will have the same shape as input IMU data here (x,y,cos(theta),sin(theta))

#     # integrating angular velocity

#     heading = torch.cumsum(imu_slice[...,2],dim=-1)*1./100. # here dt = 1 #TODO this depends on sampling rate value

#     result[...,2] = torch.cos(heading)
#     result[...,3] = torch.sin(heading)

#     # integrating the acc to get velocity
#     result[...,0] = torch.cumsum(v0[:,:S,0,None] + torch.cumsum(imu_slice[...,0]*result[...,2] - imu_slice[...,1]*result[...,3],dim=-1)*1./100.,dim=-1)*1./100.
#     result[...,1] = torch.cumsum(v0[:,:S,1,None] + torch.cumsum(imu_slice[...,0]*result[...,3] + imu_slice[...,1]*result[...,2],dim=-1)*1./100.,dim=-1)*1./100.


#     return result

In [ ]:
def double_integrate(imu_slice, v0=torch.tensor([0,0], dtype=torch.float32)):
    
    B,S,W,C = imu_slice.shape
    result = torch.zeros((B,S,W,4)) # result will have the same shape as input IMU data here (x,y,cos(theta),sin(theta))

    # integrating angular velocity

    heading = torch.cumsum(imu_slice[...,2],dim=-1)*1./100. # here dt = 1 #TODO this depends on sampling rate value

    result[...,2] = torch.cos(heading)
    result[...,3] = torch.sin(heading)


    # integrating the acc to get velocity and trajectory
    R = torch.stack([result[...,2],-result[...,3],result[...,3],result[...,2]],dim=-1).view(B,S,W,2,2)
    acc = torch.matmul(R,imu_slice[...,:2].view(B,S,W,2,1)).squeeze()

    v = v0[:,:S,None,:] + torch.cumsum( acc ,dim=-2)*1./100.

    result[...,:2] = torch.cumsum(v,dim=-2)*1./100.

    return result

In [ ]:
# reconstructed = double_integrate(imu_slice,gt_velocity[1])

# plt.figure()
# plt.plot(traj_segment[:,0],traj_segment[:,1])
# plt.quiver(traj_segment[::10,0], traj_segment[::10,1], traj_segment[::10,2],traj_segment[::10,3])


# plt.plot(reconstructed[:,0],reconstructed[:,1])
# plt.quiver(reconstructed[::10,0], reconstructed[::10,1], reconstructed[::10,2],reconstructed[::10,3])
# plt.grid()
# plt.axis('equal')
# plt.show()

# Setting Theseus optimization variables, cost functions and layer

In [ ]:
# spline motion + working from slices
# input of the model is acc_x, acc_y and omega_z in sensor frame
# output has the same size
model = SimpleNN(window*3, 3*window, hid_size=300)
model.train()
model

Number of optimization nodes in each trajectory is:

In [ ]:
N = n_pts_spline_segment*n_control_points // window
N

Now we set all optimization variables, gt and cost functions in theseus

In [ ]:
# optimization variables for inner loop
poses : List[th.SE2] = []

# number of poses is the same as the number of nodes in trajectory
for i in range(N):
    poses.append(th.SE2(torch.zeros(B,3),name=f"pose_{i}"))

Defining cost functions

In [ ]:
cost_functions = []

# adding cost factors for odometry
for i in range(N-1):
    # odometry measurmenets will depend on NN output
    # pred_acc = model(input_acc[:,i].view(B,-1))  #<====== here we attach computational graph of NN to all the odometry factors via predicting acceleration
    meas_tensor = th.SE2(torch.zeros((B,3)), name=f"predicted_odometry_{i}")

    # meas_tensor = th.SE2(torch.tensor([gt_traj[i+1] - gt_traj[i], 0, 0]).reshape(1,-1))
    cost_between = th.ScaleCostWeight(torch.ones((B,1)), name=f"scale_between_{i}")
    cost_functions.append(
                th.Between(poses[i], poses[i+1], meas_tensor,
                        cost_between,
                        name=f"between_{i}"))
    
# adding cost fuctors for absolute position
for i in range(N):
    gt_pose_tensor = th.SE2(torch.zeros((B,3)), name=f"gt_pose_{i}")
    scale_gps = th.ScaleCostWeight(torch.ones((B,1)), name=f"scale_gps_{i}")
    cost_functions.append(th.Difference(poses[i], gt_pose_tensor, scale_gps, name=f"gps_{i}"))


Adding all the cost functions to the objective of inner-loop

In [ ]:
objective = th.Objective()
for cost in cost_functions:
    objective.add(cost)

Checking what aux and optim variable the objective has:

In [ ]:
print("+"*40)
print(f"AUX variables: {len(objective.aux_vars)}")
print("+"*40)
for c in objective.aux_vars:
    print(c)
print("+"*40)

print(f"Optimization variables: {len(objective.optim_vars)}")
print("+"*40)
for c in objective.optim_vars:
    print(c)
print("+"*40)


Defining optimizer for inner loop and theseus layer 

In [ ]:
optimizer = th.GaussNewton(
        objective,
        th.CholeskyDenseSolver,
        max_iterations=3,
        step_size=0.1,
    )

state_estimator = th.TheseusLayer(optimizer)

Defining helper function to update theseus inputs for every training epoch

In [ ]:
# this plotting function will update some existing figure
plt.ion()

def plot_path(optimizer_path, groundtruth_path):
    plt.cla()
    # plt.gca().axis("equal")

    # plt.xlim(-25, 25)
    # plt.ylim(-10, 40)
    B = optimizer_path.shape[0]

    for b in range(B):
        plt.plot(
            optimizer_path[b,:,0],
            optimizer_path[b,:,1],
            linewidth=2,
            linestyle="-",
            color="tab:orange",
            label="optimizer",
        )

        #TODO add heading plt.quiver for optimized trajectory


        plt.plot(
            groundtruth_path[b,:,0],
            groundtruth_path[b,:,1],
            linewidth=2,
            linestyle="-",
            color="tab:green",
            label="groundtruth",
        )

        #TODO add heading plt.quiver for gt trajectory

    plt.title(f"mean squared error {F.mse_loss(torch.tensor(optimizer_path),groundtruth_path)}")
    plt.legend()
    plt.grid()
    plt.show()
    plt.pause(1e-12)

# Creating dataloader for our training dataset

In [ ]:
train_dataloader = DataLoader(dataset=dataset, batch_size=B, shuffle=True)

In [ ]:
n_epoch = 20

model_optimizer = torch.optim.Adam(model.parameters(), lr=5e-2)

losses = []

theseus_inputs = {}

for i in range(N):
    theseus_inputs[f"pose_{i}"] = th.SE2(torch.zeros(B, 3)).tensor

with torch.autograd.detect_anomaly():
    for epoch in tqdm(range(n_epoch)):
        for i, data in enumerate(train_dataloader):

            input_acc, y, _, gt_pose, gt_velocity = data[0],data[1],data[2],data[3],data[4]
        
            model_optimizer.zero_grad()

            # preparing theseus inputs
            # we ran input acc signals through the NN and get some output acc values (NN model is our observation function)
            # after model inference we do manual integration of model output - tensor graph should preserved

            # predicted_odometry = run_model(model, input_acc)
            predicted_acc = model(input_acc.view(-1,300)).view(input_acc.shape)

            kek = double_integrate(predicted_acc,gt_velocity)
        
            # doing manual double integration here
            theseus_inputs = {}
            for j in range(N-1):
                theseus_inputs[f"predicted_odometry_{j}"] = th.SE2(tensor=torch.tensor(kek[:,j,-1,:],requires_grad=True))

            # here we update AUX variables (predicted incremental poses updated here
            # theseus_inputs['scale_between_0'] = torch.ones((B,1))
            # theseus_inputs['scale_gps_0'] = torch.ones((B,1))
            # theseus_inputs['scale_gps_1'] = torch.ones((B,1))
            # theseus_inputs['gt_pose_0'] = F.pad(gt_traj[:,0].view(B,-1), pad=(0,2))
            # theseus_inputs['gt_pose_1'] = F.pad(gt_traj[:,1].view(B,-1), pad=(0,2))

            objective.update(theseus_inputs)
            print(f"Objective error = {objective.error_metric().mean().item()}")

            # # checking that the number of variables not gowing
            # print("+"*40)
            # print(f"AUX variables: {len(objective.aux_vars)}")
            # print("+"*40)
            # for c in objective.aux_vars:
            #     print(c)
            # print("+"*40)

            # print(f"Optimization variables: {len(objective.optim_vars)}")
            # print("+"*40)
            # for c in objective.optim_vars:
            #     print(c)
            # print("+"*40)

            # inner loop optimization is here
            # inner loop optimization result will be store into the theseus_inputs dictionary (pose_i - objects - optimization variables)
            theseus_output, _ = state_estimator.forward(
                    theseus_inputs,
                    optimizer_kwargs={
                        "track_best_solution": True,
                        "verbose": epoch % 1 == 0,
                    },
                )

            # here we transform trajectory from optimizator
            optimized_path = torch.zeros((B,N,4))
            for i in range(N):
                optimized_path[:, i] = theseus_output[f"pose_{i}"]

            gt_path = torch.zeros((B,N,4))
            for i in range(N):
                gt_path[:, i] = gt_pose[:,i]

            # calculating mse_loss function between trajectories
            mse_loss = F.mse_loss(optimized_path.squeeze(), gt_path.squeeze(),reduction='none').mean(axis=0).mean()
            loss = mse_loss
            
            loss.backward(retain_graph=True) # we need to retain_grad to keep graph for gradient calculation

            # updating model weights
            model_optimizer.step()

            # saving loss values
            losses.append(loss.item())

            # visualizing the GT and optimized trajectory
            if epoch % 2 == 0:
                plot_path(optimized_path.squeeze().detach().numpy(), gt_pose)

print(losses)
    

In [ ]:
plt.semilogy(losses)
plt.title("Train losses")
plt.grid()


1D
1)  additive noise to acc, multiplicative noise to acc
2) work from slices


full SE2
SE2
NN_input: slice of acc_x[window] and gyro_z[window]
NN_output: delta_v, delta_angle (scalars)



In [ ]:
plt.plot(losses)